In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_dataset.csv')
data["date"] = pd.to_datetime(data["date"])
data = data.set_index("date")
data

,time,target,temp,temp_pred,weather_pred,weather_fact
date,,,,,,
2023-04-01,0,479.282,5.7,5.0,"пасм, 58%","пасм, морось"
2023-04-01,1,445.182,5.7,5.0,"пасм, 58%","пасм, морось"
2023-04-01,2,424.225,5.7,5.0,"пасм, 58%","пасм, морось"
2023-04-01,3,413.866,5.0,4.0,"пасм, 71% дождь","пасм, дымка"
2023-04-01,4,408.146,5.0,4.0,"пасм, 71% дождь","пасм, дымка"
...,...,...,...,...,...,...
2023-07-31,19,453.173,17.2,18.0,"пасм, 24% дождь",п/обл
2023-07-31,20,446.287,17.2,18.0,"пасм, 24% дождь",п/обл
2023-07-31,21,444.373,15.8,17.0,"пасм, 24% дождь",п/обл


In [ ]:
from collections import Counter as Counter
import re

def see_word_count(df_col,split_delim = "[, ]+"):
  """
  Функция для оценки слов, которые можно выбрать для парсинга. Считает количество вхождений всех слов между делимитерами
  :df_col: - колонка датафрейма
  :split_delim: - по умолчанию '[, ]+' ,делит по 1 или более(если идут подряд) запятой и пробелу. Этот аргумент вставляется как паттерн регулярки для разделения строки
  """
  count_dict = Counter()
  df_col.apply(str).str.lower().apply(lambda row: re.split(f"{split_delim}",row)).apply(count_dict.update)
  return count_dict

def line_print_dict_by_value(some_dict):
  for word,count in sorted(some_dict.items(),key=lambda x : x[1],reverse=True):
    print(f"{word}:{count}")
print("Колонка с фактической погодой:")
fact_weather_word_count = see_word_count(data["weather_fact"],split_delim=",")
line_print_dict_by_value(fact_weather_word_count)
# print("\nКолонка с предсказанной погодой:")
# pred_wether_word_count =  see_word_count(data["weather_pred"])
# line_print_dict_by_value(pred_wether_word_count)



In [7]:
data["target"].describe()


count    2928.000000
mean      419.467047
std        71.393070
min       264.819000
25%       371.384250
50%       431.477500
75%       462.414250
max       607.474000
Name: target, dtype: float64

In [15]:
data.apply(lambda col: col.unique())

time            [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
target          [479.282, 445.182, 424.225, 413.866, 408.146, ...
temp            [5.7, 5.0, 4.8, 6.5, 5.9, 4.5, 3.2, 2.3, 2.2, ...
temp_pred       [5.0, 4.0, 2.0, 1.0, 6.0, -1.0, 3.0, 0.0, -3.0...
weather_pred    [пасм, 58%, пасм, 71% дождь, пасм, 46% дождь, ...
weather_fact    [пасм, морось, пасм, дымка, пасмурно, ветерок,...
dtype: object

In [ ]:
def parse_weather_column(keywords:list,src_column_name:str,target_column_name:str,df):
  '''
  Функция для парсинга отдельных столбцов, ищет вхождение переданных ключевых слов в строке и пишет в новую колонку найденное  слово или пустую строку
  :keywords: -список ключевых слов. Ищутся по порядку перебором , поэтому лучше указывать сначала длинные слова, а потом их подстроки
  (например, если первым ключом будет "ветер", то вместо "ветерок" будет найден "ветер")
  :src_column_name: - название столбца, в котором идет поиск
  :target_column_name: - желаемое название нового столбца
  :df: - объект датафрейма, в котором  исходная колонка
  return копия датафрейма с добавленным новым столбцом(функция не inplace, перезаписывайте при надобности свой датафрейм)
  '''
  def find_kword(row_string):
      nonlocal keywords
      for k in keywords:
        if k in row_string:
          return k
      return ""
  found_list = [find_kword(row) for row in df[src_column_name] ]
  new_df = df.copy()
  new_df[target_column_name] = found_list
  return new_df

updated_wind  = parse_weather_column(["ветерок","ветрище","ветер"],"weather_fact","wind",data)

updated_wind.sample(10)

,time,target,temp,temp_pred,weather_pred,weather_fact,wind
date,,,,,,,
2023-04-29,6,362.546,6.4,9.0,"обл, неб.ветер",облачно,
2023-05-13,21,426.845,12.4,12.0,обл с пр,обл с пр,
2023-07-24,1,337.707,14.7,18.0,"обл с пр, 54% дождь","пасм, дождь",
2023-04-01,0,479.282,5.7,5.0,"пасм, 58%","пасм, морось",
2023-06-02,17,430.204,11.8,13.0,пер.обл.,"малообл., ветер, очень холодно",ветер
2023-04-04,14,604.014,2.8,2.0,"пасм, 51% снег, ветер","облачно, ветерок",ветерок
2023-04-01,20,576.638,3.2,2.0,"пасм, 9% дождь","пасм, дождь",
2023-05-18,11,473.956,14.8,16.0,пасмурно,ясно,
2023-04-03,6,432.783,1.2,2.0,п/обл,"ясно, ветер",ветер


In [ ]:
updated_wind[["weather_fact","wind"]].sample(10)

,weather_fact,wind
date,,
2023-07-27,п/обл,
2023-07-19,облачно,
2023-05-28,малообл.,
2023-07-11,дымка,
2023-07-02,"дождь, ветер",ветер
2023-04-30,"п/обл, ветер",ветер
2023-06-14,"пасм, кр.дождь",
2023-05-26,"п/обл, ветер",ветер
2023-04-27,"п/обл, лед.крупа",


In [ ]:
wind_map = {
    "": 0,
    "ветерок": 0.5,
    "ветер": 1,
    "ветрище":1.5
}
updated_wind["wind"]=updated_wind["wind"].map(wind_map)
updated_wind[["weather_fact","wind"]].sample(10)

,weather_fact,wind
date,,
2023-07-25,"пасмурно, ветерок",0.5
2023-05-21,"ясно, ветер",1.0
2023-05-07,"п/обл, ветерок",0.5
2023-05-05,ясно,0.0
2023-04-16,"пасм, слаб.дождь",0.0
2023-05-15,облачно,0.0
2023-07-01,дождь,0.0
2023-07-13,дымка,0.0
2023-04-03,"ясно, ветер",1.0
